In [2]:
import pandas as pd

df = pd.read_csv("/Creditcard_data.csv")

print("Shape:", df.shape)
df.head()


Shape: (772, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
df['Class'].value_counts()


,count
Class,
0,763
1,9


In [5]:
# Separate features and target
X = df.drop('Class', axis=1)
y = df['Class']

print(X.shape, y.shape)


(772, 30) (772,)


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

print("Train class distribution:")
print(y_train.value_counts())

print("\nTest class distribution:")
print(y_test.value_counts())


Train class distribution:
Class
0    534
1      6
Name: count, dtype: int64

Test class distribution:
Class
0    229
1      3
Name: count, dtype: int64


In [7]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(X_train, y_train)

print("After Random Under-Sampling:")
print(y_rus.value_counts())


After Random Under-Sampling:
Class
0    6
1    6
Name: count, dtype: int64


In [8]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X_train, y_train)

print("After Random Over-Sampling:")
print(y_ros.value_counts())


After Random Over-Sampling:
Class
0    534
1    534
Name: count, dtype: int64


In [9]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

print("After SMOTE:")
print(y_smote.value_counts())


After SMOTE:
Class
0    534
1    534
Name: count, dtype: int64


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression(max_iter=1000)

# Train on SMOTE data
model.fit(X_smote, y_smote)

# Test on original test set
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.99      0.95      0.97       229
           1       0.00      0.00      0.00         3

    accuracy                           0.94       232
   macro avg       0.49      0.47      0.48       232
weighted avg       0.97      0.94      0.95       232



/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Scale features
X_train_scaled = scaler.fit_transform(X_smote)
X_test_scaled = scaler.transform(X_test)


In [12]:
model = LogisticRegression(max_iter=3000)

model.fit(X_train_scaled, y_smote)

y_pred = model.predict(X_test_scaled)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.99      0.96      0.97       229
           1       0.09      0.33      0.14         3

    accuracy                           0.95       232
   macro avg       0.54      0.64      0.56       232
weighted avg       0.98      0.95      0.96       232



In [13]:
y_prob = model.predict_proba(X_test_scaled)[:, 1]

# Lower threshold for fraud detection
y_pred_custom = (y_prob >= 0.3).astype(int)

print(classification_report(y_test, y_pred_custom))


              precision    recall  f1-score   support

           0       1.00      0.92      0.95       229
           1       0.10      0.67      0.17         3

    accuracy                           0.91       232
   macro avg       0.55      0.79      0.56       232
weighted avg       0.98      0.91      0.94       232



In [14]:
# Result with default threshold (0.5)
print("Logistic Regression with default threshold:")
print(classification_report(y_test, y_pred))

# Result with lowered threshold (0.3)
print("Logistic Regression with lowered threshold (0.3):")
print(classification_report(y_test, y_pred_custom))


Logistic Regression with default threshold:
              precision    recall  f1-score   support

           0       0.99      0.96      0.97       229
           1       0.09      0.33      0.14         3

    accuracy                           0.95       232
   macro avg       0.54      0.64      0.56       232
weighted avg       0.98      0.95      0.96       232

Logistic Regression with lowered threshold (0.3):
              precision    recall  f1-score   support

           0       1.00      0.92      0.95       229
           1       0.10      0.67      0.17         3

    accuracy                           0.91       232
   macro avg       0.55      0.79      0.56       232
weighted avg       0.98      0.91      0.94       232

